In [ ]:
import os
from PIL import Image
import xml.etree.ElementTree as ET
from tqdm import tqdm
from joblib import Parallel, delayed

def process_image_xml(image_file, image_dir, xml_dir):
    # Open image and get dimensions
    img = Image.open(os.path.join(image_dir, image_file))
    width, height = img.size
    depth = len(img.getbands())  # depth is the number of color channels

    # Parse the corresponding XML file
    xml_file = os.path.splitext(image_file)[0] + '.xml'
    tree = ET.parse(os.path.join(xml_dir, xml_file))
    root = tree.getroot()

    # Change width, height and depth values
    size_elem = root.find('size')
    size_elem.find('width').text = str(width)
    size_elem.find('height').text = str(height)
    size_elem.find('depth').text = str(depth)

    # Write back the changes to XML file
    tree.write(os.path.join(xml_dir, xml_file))

image_dir = 'pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Downsampled_NoAugmentation/JPEGImages'
xml_dir = 'pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Downsampled_NoAugmentation/Annotations/'

# get the list of all image file names
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

# Use Joblib to parallelize the process
Parallel(n_jobs=-1)(delayed(process_image_xml)(image_file, image_dir, xml_dir) for image_file in tqdm(image_files, desc="Processing images"))

Processing images:  42%|███████████████████████████▎                                     | 60928/144888 [00:30<00:35, 2393.26it/s]